<a href="https://colab.research.google.com/github/Eunvivi/CleanSeoulPM/blob/main/Preprocessing_tow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install koreanize-matplotlib

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os

if os.name == 'posix':
    plt.rc("font", family="AppleGothic")
else:
    plt.rc("font", family="Malgun Gothic")
plt.rc("axes", unicode_minus=False)

import koreanize_matplotlib
%config InlineBackend.figure_format = 'retina'

# 경고 무시
import warnings
warnings.filterwarnings('ignore')

# row 생략 없이 출력
pd.set_option('display.max_rows', None)
# col 생략 없이 출력
pd.set_option('display.max_columns', None)

# 1. 파일 로드

In [ ]:
data202301 = pd.read_csv('/content/drive/MyDrive/서울시공공데이터 /data/서울특별시_전동킥보드_견인_현황_20230131.csv', encoding ='CP949')
data202301.head(3)

,번호,신고일,구정보,주소,유형,조치일
0,1,2021-07-15,영등포구,서울특별시 영등포구 여의도동 84-10,보도와 차도가 구분된 도로의 차도,2021-07-15
1,2,2021-07-15,영등포구,서울특별시 영등포구 영등포동1가 97-4,보도 중앙,2021-07-15
2,3,2021-07-15,동작구,서울특별시 동작구 노량진동 231-2,보도와 차도가 구분된 도로의 차도,2021-07-15


In [ ]:
data202308 = pd.read_csv('/content/drive/MyDrive/서울시공공데이터 /data/서울특별시_전동킥보드_견인_현황_20230831.csv', encoding ='CP949')
data202308.head(3)

,번호,신고일,구정보,주소,유형,조치일
0,1,2021-07-15,영등포구,서울특별시 영등포구 여의도동 84-10,보도와 차도가 구분된 도로의 차도,2021-07-15
1,2,2021-07-15,영등포구,서울특별시 영등포구 영등포동1가 97-4,보도 중앙,2021-07-15
2,3,2021-07-15,동작구,서울특별시 동작구 노량진동 231-2,보도와 차도가 구분된 도로의 차도,2021-07-15


In [ ]:
data202312 = pd.read_excel('/content/drive/MyDrive/서울시공공데이터 /data/서울특별시_전동킥보드_견인_현황_20231231.xlsx')
data202312.head(3)

# 0번째 행 컬럼으로
# 첫번째 컬럼 삭제
# 신고일, 조치일 시간제거

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,NaN,번호,신고일,구정보,주소,유형,조치일
1,NaN,1,2023-09-01 00:00:00,도봉구,서울특별시 도봉구 쌍문동 482-1,"점자블록, 엘리베이터 입구",2023-09-01 00:00:00
2,NaN,2,2023-09-01 00:00:00,강동구,서울특별시 강동구 성내동 478,"버스정류장, 택시 승강장",2023-09-01 00:00:00


## 1-1. 2023년 12월 데이터 전처리
* 0번째 행 컬럼으로
* 첫번째 컬럼 삭제
* 신고일, 조치일 시간제거

In [ ]:
data202312.columns = data202312.iloc[0]
data202312 = data202312.drop(0)
data202312 = data202312.iloc[:, 1:]

In [ ]:
# 시간 전처리
data202312['신고일'] = pd.to_datetime(data202312['신고일'])
data202312['조치일'] = pd.to_datetime(data202312['조치일'])

data202312['신고일'] = data202312['신고일'].dt.strftime('%Y-%m-%d')
data202312['조치일'] = data202312['조치일'].dt.strftime('%Y-%m-%d')

In [ ]:
data202312.head()

,번호,신고일,구정보,주소,유형,조치일
1,1,2023-09-01,도봉구,서울특별시 도봉구 쌍문동 482-1,"점자블록, 엘리베이터 입구",2023-09-01
2,2,2023-09-01,강동구,서울특별시 강동구 성내동 478,"버스정류장, 택시 승강장",2023-09-01
3,3,2023-09-01,성동구,서울특별시 성동구 성수동2가 277-25,"횡단보도, 산책로 등",2023-09-01
4,4,2023-09-01,강동구,서울특별시 강동구 성내동 478,"버스정류장, 택시 승강장",2023-09-01
5,5,2023-09-01,마포구,서울특별시 마포구 도화동 81,보도와 차도가 구분된 도로의 차도,2023-09-01


## 1-2. 데이터프레임 병합 및 결측치 확인
* 결측치 2건 확인
* 번호 '2816'으로 중복 및 유형, 조치일 NaN
* 결측치 삭제 조치

In [ ]:
# 데이터프레임 병합
data = pd.concat([data202301, data202308, data202312])

# 병합된 데이터프레임의 인덱스 재설정
data.reset_index(drop=True, inplace=True)

In [ ]:
data = data.sort_values(by='조치일', ascending=False)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 217492 entries, 217491 to 91129
Data columns (total 6 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   번호      217492 non-null  object
 1   신고일     217492 non-null  object
 2   구정보     217492 non-null  object
 3   주소      217492 non-null  object
 4   유형      217490 non-null  object
 5   조치일     217490 non-null  object
dtypes: object(6)
memory usage: 11.6+ MB


In [ ]:
# null 데이터만 출력
null_data = data[data.isnull().any(axis=1)]
null_data
# 이상치 데이터 2건 >> 삭제 진행

,번호,신고일,구정보,주소,유형,조치일
2815,2816,2021-08-17,관악구,서울특별시 관악구 신림동 1467-10,NaN,NaN
91129,2816,2021-08-17,관악구,서울특별시 관악구 신림동 1467-10,NaN,NaN


In [ ]:
#결측치 데이터 삭제
df = data.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 217490 entries, 217491 to 0
Data columns (total 6 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   번호      217490 non-null  object
 1   신고일     217490 non-null  object
 2   구정보     217490 non-null  object
 3   주소      217490 non-null  object
 4   유형      217490 non-null  object
 5   조치일     217490 non-null  object
dtypes: object(6)
memory usage: 11.6+ MB


# 2. 주소 분리
* 서울시, 자치구, 행정동으로 분리
* 행정동 > 동 혹은 가로 끝남 주의
* 서울시, 자치구 제거

In [ ]:
import re

In [ ]:
# 주소 분리 정규식
def extract_address_parts(address):
    pattern = r"(?P<city>[^ ]+시) (?P<district>[^ ]+구) (?P<town>[^ ]+(동[0-9가]*|동|가|길|로|역)) ?(?P<landmark>산[0-9\-]*)? ?(?P<house_number>[0-9\-]+)?"
    match = re.match(pattern, address)
    if match:
        return (
            match.group('city'),
            match.group('district'),
            match.group('town'),
            match.group('landmark') if match.group('landmark') else "",
            match.group('house_number') if match.group('house_number') else ""
        )
    return None, None, None, None, None

# 새 컬럼 추가
df[['시', '구', '행정동', '지형지물', '번지']] = df['주소'].apply(lambda x: pd.Series(extract_address_parts(x)))

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 217490 entries, 217491 to 0
Data columns (total 11 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   번호      217490 non-null  object
 1   신고일     217490 non-null  object
 2   구정보     217490 non-null  object
 3   주소      217490 non-null  object
 4   유형      217490 non-null  object
 5   조치일     217490 non-null  object
 6   시       217487 non-null  object
 7   구       217487 non-null  object
 8   행정동     217487 non-null  object
 9   지형지물    217487 non-null  object
 10  번지      217487 non-null  object
dtypes: object(11)
memory usage: 19.9+ MB


In [ ]:
df.head()

,번호,신고일,구정보,주소,유형,조치일,시,구,행정동,지형지물,번지
217491,5451,2023-09-18,마포구,서울특별시 마포구 서교동 343-11,보도와 차도가 구분된 도로의 차도,2023-10-05,서울특별시,마포구,서교동,,343-11
217490,5450,2023-09-27,강남구,서울특별시 강남구 역삼동 801-5,보도 중앙,2023-10-04,서울특별시,강남구,역삼동,,801-5
217377,5337,2023-09-27,송파구,서울특별시 송파구 방이동 222-13,보도와 차도가 구분된 도로의 차도,2023-09-27,서울특별시,송파구,방이동,,222-13
217385,5345,2023-09-27,강북구,서울특별시 강북구 미아동 231,"횡단보도, 산책로 등",2023-09-27,서울특별시,강북구,미아동,,231
217384,5344,2023-09-27,송파구,서울특별시 송파구 석촌동 314,보도와 차도가 구분된 도로의 차도,2023-09-27,서울특별시,송파구,석촌동,,314


In [ ]:
df['시'].value_counts()

서울특별시    217449
서울시          38
Name: 시, dtype: int64

In [ ]:
print(df['구'].value_counts())
print('-'*20)
print(df['구정보'].value_counts())

마포구     22235
송파구     21188
성동구     18435
강남구     17899
영등포구    17557
서초구     14649
동작구     12122
관악구     11005
동대문구     9202
강서구      8942
용산구      8870
광진구      6968
강동구      6420
중랑구      6169
노원구      5321
도봉구      5205
서대문구     4894
중구       4139
종로구      3555
강북구      3252
성북구      3016
구로구      2976
양천구      2681
은평구       774
금천구        13
Name: 구, dtype: int64
--------------------
마포구     22235
송파구     21186
성동구     18435
강남구     17897
영등포구    17557
서초구     14649
동작구     12123
관악구     11005
동대문구     9204
강서구      8942
용산구      8870
광진구      6968
강동구      6420
중랑구      6169
노원구      5321
도봉구      5205
서대문구     4892
중구       4143
종로구      3555
강북구      3252
성북구      3018
구로구      2976
양천구      2681
은평구       774
금천구        13
Name: 구정보, dtype: int64


In [ ]:
df['행정동'].value_counts()

성수동2가        10602
서초동           9816
봉천동           7993
대치동           6080
여의도동          5621
서교동           5038
문정동           4647
성수동1가         3980
가락동           3772
장안동           3629
사당동           3608
신길동           3356
삼성동           3306
동교동           3305
역삼동           3125
양재동           2797
신림동           2568
상도동           2423
구로동           2412
방이동           2388
면목동           2315
논현동           2187
송파동           2142
등촌동           2046
잠실동           2008
상계동           2000
장지동           1906
성내동           1899
방학동           1888
망원동           1808
자양동           1755
오금동           1746
합정동           1725
창동            1608
대림동           1565
방배동           1538
가양동           1521
화곡동           1510
목동            1481
대방동           1477
공덕동           1419
신대방동          1407
염창동           1315
답십리동          1311
행당동           1311
마곡동           1310
미아동           1291
신사동           1288
쌍문동           1287
군자동           1273
화양동           1251
수유동           1161
효창동         

In [ ]:
df['지형지물'].value_counts()

            215970
산150-127       159
산25-33          67
산11-1           49
산21-15          45
산21-9           41
산17-7           38
산31-37          31
산60-1           31
산10-27          26
산113-44         22
산1-1            21
산49-142         19
산17-15          19
산150-124        18
산75-2           18
산6              17
산5-68           14
산2-1            14
산77-34          13
산11-23          13
산76-34          12
산2-73           12
산50-7           12
산31-33          12
산1-4            12
산4-4            11
산1-77           11
산150-126        11
산206-6          10
산103-52         10
산77-22          10
산130-18          9
산65-169          8
산227-6           8
산77-1            8
산8-13            8
산148-9           8
산127-63          8
산101-42          8
산65-167          7
산63-2            7
산10-5            7
산2-75            6
산5-56            6
산20-2            6
산88-1            6
산32-82           6
산1-47            6
산56-3            6
산10-24           6
산18-4            6
산52-14      

In [ ]:
df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 217487 entries, 217491 to 0
Data columns (total 11 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   번호      217487 non-null  object
 1   신고일     217487 non-null  object
 2   구정보     217487 non-null  object
 3   주소      217487 non-null  object
 4   유형      217487 non-null  object
 5   조치일     217487 non-null  object
 6   시       217487 non-null  object
 7   구       217487 non-null  object
 8   행정동     217487 non-null  object
 9   지형지물    217487 non-null  object
 10  번지      217487 non-null  object
dtypes: object(11)
memory usage: 19.9+ MB


In [ ]:
df.head()

,번호,신고일,구정보,주소,유형,조치일,시,구,행정동,지형지물,번지
217491,5451,2023-09-18,마포구,서울특별시 마포구 서교동 343-11,보도와 차도가 구분된 도로의 차도,2023-10-05,서울특별시,마포구,서교동,,343-11
217490,5450,2023-09-27,강남구,서울특별시 강남구 역삼동 801-5,보도 중앙,2023-10-04,서울특별시,강남구,역삼동,,801-5
217377,5337,2023-09-27,송파구,서울특별시 송파구 방이동 222-13,보도와 차도가 구분된 도로의 차도,2023-09-27,서울특별시,송파구,방이동,,222-13
217385,5345,2023-09-27,강북구,서울특별시 강북구 미아동 231,"횡단보도, 산책로 등",2023-09-27,서울특별시,강북구,미아동,,231
217384,5344,2023-09-27,송파구,서울특별시 송파구 석촌동 314,보도와 차도가 구분된 도로의 차도,2023-09-27,서울특별시,송파구,석촌동,,314


In [ ]:
#필요한 컬럼만
tow = df[['번호','신고일','조치일','구','행정동','유형']]

In [ ]:
#도로명 주소 행정동으로 수정
tow['행정동'] = tow['행정동'].replace({
    '장승배기로': '노량진동',
    '노량진로': '노량진동',
    '사당동139': '사당동',
    '퇴계로': '묵정동',
    '북촌로': '계동',
    '창덕궁3가길': '원서동',
    '상도로60길': '상도동',
    '평창문화로': '평창동',
    '을지로': '을지로2가',
    '계동길': '계동',
    '가락로36길': '방이동',
    '상도로15길': '상도동',
    '왕십리로33길': '하왕십리동',
    '장승배기로13길': '노량진동',
    '마천로': '오금동',
    '사당동1131': '사당동',
    '창덕궁1길': '계동',
    '동호로17길': '신당동',
    '장군봉1길': '봉천동',
    '관악로28길': '봉천동',
    '등용로14길': '대방동',
    '여의대방로22나길': '신대방동',
    '신대방3길': '신대방동',
    '남부순환로': '신림동',
    '청파로': '청파동3가',
    '사당동68': '사당동',
    '백제고분로': '석촌동',
    '사당동205': '사당동',
    '장승배기로27길': '노량진동',
    '중림로8길': '중림동',
    '지봉로13길': '창신동',
    '창덕궁3길': '원서동',
    '새문안로3길': '내수동',
    '상도로': '상도동',
    '지봉로12길': '숭인동',
    '명륜길': '명륜3가',
    '필운대로': '필운동',
    '사당로13길': '사당동',
    '명륜5길': '명륜3가',
    '자양로18길': '구의동',
    '삼청로7길': '팔판동',
    '혜화로16길': '혜화동',
    '돈화문로8길': '묘동',
    '성균관로5길': '명륜3가',
    '삼청로': '삼청동',
    '성균관로': '명륜3가',
    '고척로27마길': '개봉동',
    '사당로': '사당동',
    '사직로': '사직동',
    '평창1길': '평창동',
    '평창길': '평창동',
    '장승배기로10가길': '상도동',
    '낙산성곽서길': '동숭동',
    '등용로': '상도동',
    '대림로': '대림동',
    '새문안로': '신문로2가',
    '반포대로': '반포동',
    '동숭4가길': '동숭동',
    '동작대로43길': '동작동',
    '상도역': '상도1동',
    '옥인길': '누상동',
    '성균관로14길': '명륜1가',
    '낙산성곽동길': '창신동',
    '아차산로': '광장동',
    '동호로': '금호동3가',
    '창경궁로': '원남동',
    '평창20길': '평창동',
    '동망산길': '숭인동',
    '옥인6가길': '누상동',
    '명륜3길': '명륜3가',
    '통일로8길': '송월동',
    '종로1길': '수송동',
    '사직로9길': '사직동',
    '명륜7길': '명륜3가',
    '인사동9길': '인사동',
    '낙산길': '삼선동2가',
    '동순라길': '숭인동',
    '북촌로11다길': '삼청동',
    '대학로': '동숭동',
    '평창11길': '평창동',
    '장문로6길': '동빙고동',
    '종로31길': '연지동',
    '혜화로11길': '혜화동',
    '자하문로19길': '옥인동',
    '필운대로1길': '필운동',
    '노량진로32길': '노량진동'
})

In [ ]:
#전처리 완료
tow.head()

,번호,신고일,조치일,구,행정동,유형
217491,5451,2023-09-18,2023-10-05,마포구,서교동,보도와 차도가 구분된 도로의 차도
217490,5450,2023-09-27,2023-10-04,강남구,역삼동,보도 중앙
217377,5337,2023-09-27,2023-09-27,송파구,방이동,보도와 차도가 구분된 도로의 차도
217385,5345,2023-09-27,2023-09-27,강북구,미아동,"횡단보도, 산책로 등"
217384,5344,2023-09-27,2023-09-27,송파구,석촌동,보도와 차도가 구분된 도로의 차도


# 3. 행정동 코드 추가

In [ ]:
#서울시 행정동 데이터 로드
scode =pd.read_csv('/content/drive/MyDrive/서울시공공데이터 /data/서울시 건축물대장 법정동 코드정보.csv', encoding='CP949')
slo =pd.read_csv('/content/drive/MyDrive/서울시공공데이터 /data/bjd_info_except_boundary.csv', encoding='CP949')

In [ ]:
scode.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2355 entries, 0 to 2354
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   시군구코드   2355 non-null   int64 
 1   법정동코드   2355 non-null   int64 
 2   행정동코드   2355 non-null   int64 
 3   시도명     2355 non-null   object
 4   시군구명    2354 non-null   object
 5   법정동명    2355 non-null   object
 6   행정동명    1243 non-null   object
 7   적용시작일   2355 non-null   int64 
 8   적용만료일   2355 non-null   int64 
dtypes: int64(5), object(4)
memory usage: 165.7+ KB


In [ ]:
# '행정동코드' 추가
for index, row in tow.iterrows():
    match_scode = scode[(scode['행정동명'] == row['행정동']) | (scode['법정동명'] == row['행정동'])]
    if not match_scode.empty:
        tow.loc[index, '법정동코드'] = match_scode.iloc[0]['법정동코드']

In [ ]:
tow = tow[['번호','신고일','조치일','구','행정동','유형','법정동코드']]

In [ ]:
tow['법정동코드'] = tow['법정동코드'].astype(int)

In [ ]:
tow.head()

,번호,신고일,조치일,구,행정동,유형,법정동코드
217491,5451,2023-09-18,2023-10-05,마포구,서교동,보도와 차도가 구분된 도로의 차도,11000
217490,5450,2023-09-27,2023-10-04,강남구,역삼동,보도 중앙,10100
217377,5337,2023-09-27,2023-09-27,송파구,방이동,보도와 차도가 구분된 도로의 차도,11100
217385,5345,2023-09-27,2023-09-27,강북구,미아동,"횡단보도, 산책로 등",10100
217384,5344,2023-09-27,2023-09-27,송파구,석촌동,보도와 차도가 구분된 도로의 차도,500


In [ ]:
# slo.head()

In [ ]:
# '행정동코드' 추가
for index, row in tow.iterrows():
    match_slo = slo[(slo['emd_nm'] == row['행정동'])]
    if not match_scode.empty:
        tow.loc[index, '법정동코드'] = match_scode.iloc[0]['법정동코드']

,bjd_cd,center_point,center_long,center_lati,bjd_nm,level,sd_cd,sd_nm,sgg_cd,sgg_nm,emd_cd,emd_nm,li_nm,sgg_cnt,emd_cnt,li_cnt,sgg_list,emd_list,li_list
474,1171010500,POINT (127.10251312460893 37.502726254178654),127.102513,37.502726,서울특별시 송파구 석촌동,2,11,서울특별시,11710.0,송파구,11710105,석촌동,NaN,0,0,0,NaN,NaN,NaN


In [ ]:
# '행정동'과 'emd_nm'이 일치하는 행 찾기
merged_tow = pd.merge(tow, slo, how='left', left_on='행정동', right_on='emd_nm')

# 'center_long'와 'center_lati' 컬럼 추가
merged_tow['중심경도'] = merged_tow['center_long']
merged_tow['중심위도'] = merged_tow['center_lati']

# 필요한 컬럼만 선택
merged_tow = merged_tow[['번호', '신고일', '조치일', '구', '행정동', '유형', '중심경도', '중심위도']]

# 컬럼명 한국어로 변경
merged_tow.columns = ['번호', '신고일', '조치일', '구', '행정동', '유형', '중심경도', '중심위도']

In [ ]:
merged_tow.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 335728 entries, 0 to 335727
Data columns (total 8 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   번호      335728 non-null  object 
 1   신고일     335728 non-null  object 
 2   조치일     335728 non-null  object 
 3   구       335728 non-null  object 
 4   행정동     335728 non-null  object 
 5   유형      335728 non-null  object 
 6   중심경도    335728 non-null  float64
 7   중심위도    335728 non-null  float64
dtypes: float64(2), object(6)
memory usage: 23.1+ MB


# 5. 중복데이터 삭제

In [ ]:
#중복 데이터 확인
dup_tow = merged_tow[merged_tow.duplicated(keep=False)]
dup_tow .info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 274656 entries, 60881 to 335727
Data columns (total 8 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   번호      274656 non-null  object 
 1   신고일     274656 non-null  object 
 2   조치일     274656 non-null  object 
 3   구       274656 non-null  object 
 4   행정동     274656 non-null  object 
 5   유형      274656 non-null  object 
 6   중심경도    274656 non-null  float64
 7   중심위도    274656 non-null  float64
dtypes: float64(2), object(6)
memory usage: 18.9+ MB


In [ ]:
# 중복 데이터 제거 (첫 번째 데이터만 남김)
fin_tow = merged_tow.drop_duplicates(keep='first')

In [ ]:
fin_tow.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 198400 entries, 0 to 335678
Data columns (total 8 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   번호      198400 non-null  object 
 1   신고일     198400 non-null  object 
 2   조치일     198400 non-null  object 
 3   구       198400 non-null  object 
 4   행정동     198400 non-null  object 
 5   유형      198400 non-null  object 
 6   중심경도    198400 non-null  float64
 7   중심위도    198400 non-null  float64
dtypes: float64(2), object(6)
memory usage: 13.6+ MB


In [ ]:
# 중복데이터 확인
t = fin_tow [fin_tow .duplicated(keep=False)]
t

,번호,신고일,조치일,구,행정동,유형,중심경도,중심위도


# 6. 저장

In [ ]:
fin_tow.to_csv('/content/drive/MyDrive/서울시공공데이터 /data/tow.csv', index=False, encoding='utf-8-sig')